In [2]:
import numpy as np
import matplotlib.pyplot as plt

In [8]:
def print_progress(i,max):
    progress = (i / max) * 100
    completed = int((i / max) * 50)
    bar = '#' * completed + '-' * (50 - completed)
    print(f'[{progress:3.0f}%] [{bar}]', end='\r')

In [29]:
nx = 100
ny = nx
nt = 1010
dx = 1 / (nx - 1)
dy = dx
dt = 0.0001
alpha = 0.13  # Coeficiente de difusión más alto

x = np.linspace(0, 1, nx)
y = np.linspace(0, 1, ny)

T = np.zeros((nx, ny, nt))
rho = np.ones((nx, ny))  # Densidad constante en toda la placa

X, Y = np.meshgrid(x, y)

# Definir un cilindro con temperatura constante de 0 como distribución inicial de temperatura
cylinder_center_x = 0.5
cylinder_center_y = 0.5
cylinder_radius = 0.3

cylinder_mask = (X - cylinder_center_x) ** 2 + (Y - cylinder_center_y) ** 2 <= cylinder_radius ** 2

T[:, :, 0] = (1 - cylinder_mask) * 100  # Asignar 500 a la placa y 0 al cilindro

# Condiciones de frontera igual
T[0, :, :] = 100
T[nx - 1, :, :] = 100
T[:, 0, :] = 100
T[:, ny - 1, :] = 100

for j in range(nt-1):
    for i in range(1, nx - 1):
        for k in range(1, ny - 1):
            d2T_dx2 = (T[i + 1, k, j] - 2 * T[i, k, j] + T[i - 1, k, j]) / (dx ** 2 * rho[i, k])
            d2T_dy2 = (T[i, k + 1, j] - 2 * T[i, k, j] + T[i, k - 1, j]) / (dy ** 2 * rho[i, k])

            T[i, k, j + 1] = T[i, k, j] + alpha * dt * (d2T_dx2 + d2T_dy2)
            
    #Mostrar porcentaje de avance
    print_progress(j,nt)

In [30]:
fig = plt.figure(figsize=(9,8))
ax = plt.subplot(1,1,1)

ax.set_xlabel('x')
ax.set_ylabel('y')

cont = ax.contourf(X,Y,T[:,:,0],levels = np.linspace(0,100,11))
cbar = plt.colorbar(cont)
plt.close(fig)

for i in range(0,nt,10):
    cont = ax.contourf(X,Y,T[:,:,i],levels = np.linspace(0,100,11))
    fig.suptitle(f'n = {i}, t = {i*dt:.4f}')
    fig.savefig(f'figures/{i}.jpg')
    plt.close(fig)

    print_progress(i,nt)

In [32]:
# Crear figura y eje 3D
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Graficar superficie 3D
for i in range(0,nt,10):
    ax.plot_surface(X, Y, T[:,:,i], cmap='viridis')
    fig.suptitle(f'n = {i}, t = {i*dt:.4f}')
    fig.savefig(f'figures2/{i}.jpg')
    plt.close(fig)

# Configurar etiquetas de los ejes
ax.set_xlabel('Eje X')
ax.set_ylabel('Eje Y')
ax.set_zlabel('Eje Z')

Text(0.5, 0, 'Eje Z')

Error in callback <function flush_figures at 0x7fc3d9b1fa30> (for post_execute):


KeyboardInterrupt: 

In [31]:
from PIL import Image
import os

# Ruta de la carpeta que contiene las imágenes
folder_path = 'figures/'

# Lista para almacenar las imágenes
images = []

# Obtener todas las imágenes en la carpeta
for filename in os.listdir(folder_path):
    if filename.endswith('.jpg') or filename.endswith('.png'):
        image_path = os.path.join(folder_path, filename)
        image = Image.open(image_path)
        images.append((filename, image))

# Ordenar las imágenes por el nombre de archivo
images.sort(key=lambda x: int(x[0].replace('.jpg','')))

# Guardar las imágenes como GIF
output_path = 'animacion.gif'
frames = [image for _, image in images]
frames[0].save(output_path, save_all=True, append_images=frames[1:], optimize=False, duration=200, loop=0)
